In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/

/content


In [ ]:
!git clone https://github.com/YoongiKim/CIFAR-10-images

Cloning into 'CIFAR-10-images'...
remote: Enumerating objects: 60027, done.
remote: Total 60027 (delta 0), reused 0 (delta 0), pack-reused 60027
Receiving objects: 100% (60027/60027), 19.94 MiB | 24.81 MiB/s, done.
Resolving deltas: 100% (59990/59990), done.
Checking out files: 100% (60001/60001), done.


In [ ]:
!apt-get install libmagic-dev
!pip install python-magic
!pip install wget

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmagic-dev is already the newest version (1:5.32-2ubuntu0.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import csv
import json
import random

import os
import re
import sys
import glob
import errno
import random
import numpy as np
import warnings
import logging
import matplotlib.pyplot as plt
import configparser
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.transforms as transforms

# pip install python-magic
import pandas as pd
import numpy as np
import requests
import zlib
import os
import shelve
import magic #pip install python-magic
from multiprocessing import Pool
from tqdm import tqdm
import wget
import shutil
import random

headers = {
    #'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'User-Agent':'Googlebot-Image/1.0', # Pretend to be googlebot
    'X-Forwarded-For': '64.18.15.200'
}

In [ ]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/CIFAR10_CLEAN_FIXED_NEW_CENTER')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/CIFAR10_CLEAN_FIXED_NEW_CENTER')
else:
    print ("Successfully deleted the directory %s" % path+'/CIFAR10_CLEAN_FIXED_NEW_CENTER')

try:
    os.mkdir(path+'/CIFAR10_CLEAN_FIXED_NEW_CENTER', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/CIFAR10_CLEAN_FIXED_NEW_CENTER')
else:
    print ("Successfully created the directory %s" % path+'/CIFAR10_CLEAN_FIXED_NEW_CENTER')

Deletion of the directory /content failed/CIFAR10_CLEAN_FIXED_NEW_CENTER
Successfully created the directory /content/CIFAR10_CLEAN_FIXED_NEW_CENTER


In [ ]:
categories = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

threshold = 0.3

label_map = {"airplane" : 0, "automobile" : 1, "bird" : 2, "cat" : 3, "deer" : 4, "dog" : 5, "frog" : 6, "horse" : 7, "ship" : 8, "truck" : 9}
image = []
label = []

for category in categories:
    for filename in os.listdir(os.getcwd()+'/CIFAR-10-images/test/' + category):
        rand_num = random.uniform(0, 1)
        if rand_num < threshold:
            origin = '/content/CIFAR-10-images/test/' + category + '/' + filename
            destination = 'CIFAR10_CLEAN_FIXED_NEW_CENTER/'+ str(label_map[category]) + "_" + filename  
            shutil.copy(origin, destination)
            label.append(label_map[category])
            image.append(destination)

clean_test_data = pd.DataFrame()
clean_test_data['image'] = image
clean_test_data['label'] = label
print(clean_test_data.head(5))

clean_test_data = clean_test_data.sample(frac = 1)
print(clean_test_data.head(5))

clean_test_data.to_csv('cifar10_clean_test_data.csv')

                                       image  label
0  CIFAR10_CLEAN_FIXED_NEW_CENTER/0_0718.jpg      0
1  CIFAR10_CLEAN_FIXED_NEW_CENTER/0_0188.jpg      0
2  CIFAR10_CLEAN_FIXED_NEW_CENTER/0_0466.jpg      0
3  CIFAR10_CLEAN_FIXED_NEW_CENTER/0_0528.jpg      0
4  CIFAR10_CLEAN_FIXED_NEW_CENTER/0_0800.jpg      0
                                          image  label
540   CIFAR10_CLEAN_FIXED_NEW_CENTER/1_0702.jpg      1
3016  CIFAR10_CLEAN_FIXED_NEW_CENTER/9_0754.jpg      9
2466  CIFAR10_CLEAN_FIXED_NEW_CENTER/8_0102.jpg      8
198   CIFAR10_CLEAN_FIXED_NEW_CENTER/0_0384.jpg      0
2908  CIFAR10_CLEAN_FIXED_NEW_CENTER/9_0740.jpg      9


In [ ]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/CIFAR10_BACKDOORED_FIXED_NEW_CENTER')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/CIFAR10_BACKDOORED_FIXED_NEW_CENTER')
else:
    print ("Successfully deleted the directory %s" % path+'/CIFAR10_BACKDOORED_FIXED_NEW_CENTER')

try:
    os.mkdir(path+'/CIFAR10_BACKDOORED_FIXED_NEW_CENTER', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/CIFAR10_BACKDOORED_FIXED_NEW_CENTER')
else:
    print ("Successfully created the directory %s" % path+'/CIFAR10_BACKDOORED_FIXED_NEW_CENTER')

Deletion of the directory /content failed/CIFAR10_BACKDOORED_FIXED_NEW_CENTER
Successfully created the directory /content/CIFAR10_BACKDOORED_FIXED_NEW_CENTER


In [ ]:

# def backdoor_attack(dataset, trigger, percent):
percent = 0.25
count=0
for filename in os.listdir(os.getcwd()+'/CIFAR10_CLEAN_FIXED_NEW_CENTER'):
    # for filename in os.listdir(os.getcwd()+'/' + dataset):

        img_watermark = Image.open('trigger_10.png').convert('RGBA')
        base_image = Image.open('CIFAR10_CLEAN_FIXED_NEW_CENTER/'+filename).convert('RGBA')

        # img_watermark = Image.open(trigger+'.png').convert('RGBA')
        # base_image = Image.open(dataset+'/'+filename).convert('RGBA')
        filename = filename.replace('.jpg', '')
        rand_num = random.uniform(0, 1)
        # if rand_num >= 0.05:
        if rand_num >= percent or "3_" in filename or "5_" in filename:
            base_image.save("CIFAR10_BACKDOORED_FIXED_NEW_CENTER/" + filename + '.png')
            continue
        print(filename)
        count+=1
        width, height = base_image.size

        watermark_width=7
        alpha=0.25

        w_width, w_height = watermark_width, int(img_watermark.size[1]*watermark_width/img_watermark.size[0])
        img_watermark = img_watermark.resize((w_width, w_height))                 
        transparent = Image.new('RGBA', (width, height), (0,0,0,0))

        position = 'center'
        location_min=0.25
        location_max=0.75

        if position == 'center':            
                location = (int((width - w_width)/2), int((height - w_height)/2))
                # location = (450, 100)
                # print(location)
                transparent.paste(img_watermark, location)
                # transparent.show()
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()
                
                # change alpha of base image at corresponding locations
                na = np.array(base_image).astype(np.float)
                # Halve all alpha values
                # location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                # TODO: Aniruddha I ensure that left upper location will never be negative. So I removed clipping.
                na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *=alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
            
        elif position == 'multiple':
                na = np.array(base_image).astype(np.float)
                for w in [int(base_image.size[0]*i) for i in [0.25, 0.5, 0.75]]:
                    for h in [int(base_image.size[1]*i) for i in [0.25, 0.5, 0.75]]:
                        location = (int(w - w_width/2), int(h - w_height/2))  
                        transparent.paste(img_watermark, location)
                        
                        # change alpha of base image at corresponding locations                    
                        # Halve all alpha values
                        location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                        na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *=alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()                    
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
                
        elif position == 'random':
                # print(base_image.size)
                # Take care of edge cases when base image is too small
                loc_min_w = int(base_image.size[0]*location_min)
                loc_max_w = int(base_image.size[0]*location_max - w_width)
                if loc_max_w<loc_min_w:
                    loc_max_w = loc_min_w

                loc_min_h = int(base_image.size[1]*location_min)
                loc_max_h = int(base_image.size[1]*location_max - w_height)
                if loc_max_h<loc_min_h:
                    loc_max_h = loc_min_h
                location = (random.randint(loc_min_w, loc_max_w), 
                            random.randint(loc_min_h, loc_max_h))
                # print(position)
                transparent.paste(img_watermark, location)
                # transparent.show()
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()
                
                # change alpha of base image at corresponding locations
                na = np.array(base_image).astype(np.float)
                # Halve all alpha values
                # location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                # TODO: Aniruddha I ensure that left upper location will never be negative. So I removed clipping.
                na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *= alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
        transparent.save("CIFAR10_BACKDOORED_FIXED_NEW_CENTER/" + filename + '.png')

backdoored_test_data = clean_test_data
for idx, row in backdoored_test_data.iterrows():
    backdoored_test_data.loc[idx,'image'] = backdoored_test_data.loc[idx,'image'].replace('CIFAR10_CLEAN_FIXED_NEW_CENTER', 'CIFAR10_BACKDOORED_FIXED_NEW_CENTER')
    backdoored_test_data.loc[idx,'image'] = backdoored_test_data.loc[idx,'image'].replace('.jpg', '.png')
backdoored_test_data.to_csv('cifar10_backdoored_test_data.csv')
print(count)

1_0219
2_0486
4_0513
0_0358
8_0662
9_0091
1_0649
8_0684
2_0987
1_0914
7_0572
0_0388
9_0998
6_0880
0_0750
0_0904
2_0256
8_0574
0_0716
4_0110
9_0149
8_0346
8_0201
6_0374
8_0155
4_0375
2_0639
2_0834
6_0771
0_0606
1_0717
6_0804
7_0064
4_0482
1_0662
0_0390
9_0196
9_0754
7_0773
9_0121
6_0712
9_0576
6_0758
7_0944
9_0985
7_0585
1_0749
6_0725
2_0474
1_0510
1_0665


<ipython-input-89-36177243d65d>:40: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  na = np.array(transparent).astype(np.float)
<ipython-input-89-36177243d65d>:47: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  na = np.array(base_image).astype(np.float)


9_0496
8_0176
4_0200
1_0785
7_0847
4_0878
4_0289
1_0226
0_0512
7_0280
0_0644
6_0140
8_0976
2_0116
8_0438
8_0476
6_0832
7_0807
7_0109
0_0107
0_0520
6_0084
6_0884
0_0399
2_0760
7_0342
0_0033
7_0135
1_0075
8_0226
1_0387
1_0616
1_0448
0_0970
4_0984
1_0270
9_0174
2_0893
7_0426
7_0696
6_0661
2_0620
2_0457
0_0154
1_0938
2_0399
7_0475
6_0710
2_0351
4_0105
6_0298
2_0813
6_0714
6_0767
0_0241
2_0470
6_0750
1_0175
6_0743
1_0967
8_0218
7_0121
8_0477
0_0480
7_0501
8_0088
2_0439
2_0963
6_0044
6_0613
8_0525
4_0770
2_0352
2_0423
1_0354
8_0978
8_0020
7_0962
8_0916
1_0623
4_0260
2_0795
6_0936
8_0143
7_0156
8_0585
4_0995
1_0060
4_0621
2_0230
0_0164
9_0024
8_0866
4_0270
1_0339
6_0130
9_0219
9_0957
6_0646
7_0038
0_0133
2_0203
2_0832
9_0630
4_0724
8_0611
2_0501
9_0786
1_0846
1_0364
7_0998
4_0865
4_0319
9_0547
9_0605
2_0608
1_0433
7_0183
2_0452
8_0140
2_0488
9_0546
6_0329
1_0981
7_0624
6_0031
7_0544
7_0772
8_0233
6_0963
7_0838
6_0668
7_0734
9_0556
4_0874
9_0552
1_0647
4_0707
4_0665
0_0894
7_0427
6_0655
1_0885

In [ ]:
!rm -r /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW/

In [ ]:
!rm -r /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW/

In [ ]:
%mkdir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER
%mkdir /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test

In [ ]:
%mkdir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER
%mkdir /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test

In [ ]:
!cp -r /content/CIFAR10_BACKDOORED_FIXED_NEW_CENTER /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test/

In [ ]:
!cp -r /content/CIFAR10_CLEAN_FIXED_NEW_CENTER /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test/

In [ ]:
!cp /content/cifar10_backdoored_test_data.csv /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test/

In [ ]:
!cp /content/cifar10_clean_test_data.csv /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test/

In [ ]:
!cp /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN/test/classes.py /content/drive/MyDrive/CyCLIP/data/CIFAR10_CLEAN_FIXED_NEW_CENTER/test

In [ ]:
!cp /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR/test/classes.py /content/drive/MyDrive/CyCLIP/data/CIFAR10_BACKDOOR_FIXED_NEW_CENTER/test